In [2]:
import requests
import urllib3

from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.ssl_ import create_urllib3_context

In [4]:
# Custom SSL context to allow smaller DH keys
class SSLAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        context = create_urllib3_context()
        context.set_ciphers('DEFAULT@SECLEVEL=1')  # Lowering the security level
        self.poolmanager = urllib3.PoolManager(*args, ssl_context=context, **kwargs)

In [30]:
# Fectch the website u want
def fetch_web_page(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    headers = {'User-Agent': user_agent}

    # Create a session and mount the custom SSL adapter
    session = requests.Session()
    session.mount('https://', SSLAdapter())

    # Make the request with headers
    response = session.get(url, headers=headers)
    response.encoding = 'utf-8'

    # Parse the response content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [ ]:
# Fetch all the courses' url of a department
def fetch_courses_url(url):
    # Fetching all the courses' name & url
    iim_soup = fetch_web_page(url)
    # Find all td elements containing course information
    course_blocks = iim_soup.find_all('td')

    courses_url = []
    course_list = []
    for block in course_blocks:
        if block.find('span', class_='course_name'):
            course_name = block.find('span', class_='course_name').getText().strip()
            course_url = ""
            if block.find('a'):  # Directly look for 'a' tag under 'block'
                a_tag = block.find('a')
                course_url = a_tag['href'].strip()
            # Check if the course name is already added to avoid duplicates
            if course_name and course_name not in course_list:
                course_list.append(course_name)
                if course_url: 
                    courses_url.append(course_url)
                print("Course Name:", course_name)
                print("Course Url:", course_url)
                print()

    print(course_list)
    return courses_url

In [36]:
# Fetch the syllabus url and content
def fetch_syllabus_page(url):
    course_soup = fetch_web_page(url)
    course_elements = course_soup.find_all('td')
    newest_syllabus_url = course_elements[2].find('a')['href'].strip()
    print(newest_syllabus_url)
    
    syllabus_soup = fetch_web_page(newest_syllabus_url)
    syllabus_content = syllabus_soup.find('div', id = 'content')

    syllabus_div_elements = syllabus_content.find_all('div', class_ = 'block')
    syllabus_description = syllabus_div_elements[2].getText().strip()
    print(syllabus_description)
    print('----------------------')
    print()

In [37]:
iim_course_url = 'https://course.ncku.edu.tw/index.php?c=qry11215&i=AGZTOVJhVW0OeVdyVW9VNFRqDSxValt5AG8HawxgDzZQN1JsATsPKlFnADABPAUiUzQCJQ4zCTxTZwMyAG8JJFc5UDoBNgE4DWBUOAk4UDpRLgZ2B2sEMwJsAyQAMglkASdVJ1sFVW0CYVt4DzMGcA4wVWUAPARyBTUDJlFFVDsAIFMhUm5VKg5rVztVZFU0VGANNVViW2EAZgc4DCEPdFA8UmcBOg97US4AeAF1BSJTbAJ0DjIJaFNnAyMAIAkxVzRQZQF0ASENOlRxCTNQN1FvBicHMgRrAjoDagAzCWYBMlVxW2FVcAI0W2sPMgYhDlhVcgA9BC0FYgM7UTVUMwAyUzhSNFVtDjNXO1UuVXZUag05VTlbeQAwB2cMKg9zUFlSOgFvD3tRZgByATwFMlNtAiUOQAluU38DOgAoCSJXI1A6ATcBOQ0jVHAJK1A7UTQGPwdlBD4CLQNvAGwJMQFsVTpbYFUzAjVbMw8yBjIOMVU5ADwEYQU7AzFRP1RrAD1TMVJuVT4OOFcwVWRVNVRrDWdVOVtoAG8HawxgDzZQN1JhATsPJFEuADsBNwU6U3UCYA4qCWZTPwM7AGUJZFct'
courses_url = fetch_courses_url(iim_course_url)
for url in courses_url:
    fetch_syllabus_page(url)

http://class-qry.acad.ncku.edu.tw/syllabus/syllabus.php?syear=0112&sem=2&co_no=H311220&class_code=
課程概述 Course Description
       This course examines the allocation of scarce resources and the economic reasoning used by people as consumers, producers, savers, investors, workers, voters, and as government agencies. Key elements include the study of scarcity, supply and demand, market structures, the role of government, national income determination, money and the role of financial institutions. This course examines the allocation of scarce resources and the economic reasoning used by people as consumers, producers, savers, investors, workers, voters, and as government agencies. Key elements include the study of scarcity, supply and demand, market structures, the role of government, national income determination, money and the role of financial institutions.
----------------------

http://class-qry.acad.ncku.edu.tw/syllabus/syllabus.php?syear=0112&sem=2&co_no=H341000&class_code=
課程概述 Co